In [11]:
from Loader import fileloader as fl
from Loader import datahandledatabase as dhdb
import pandas as pd
import numpy as np
import time
from datetime import datetime
import openpyxl as xlsx

In [12]:
periodo = 201808
start_date_pre = '2018-07-16' # Only DDNN1 '2018-07-16'
start_date = '2018-08-16' # Captura teams '2018-08-16'
end_date = '2018-09-16' # End of Captura teams '2018-09-15'
start_date_pre_dt = datetime.strptime(start_date_pre, '%Y-%m-%d')
start_date_dt = datetime.strptime(start_date, '%Y-%m-%d')
end_date_dt = datetime.strptime(end_date, '%Y-%m-%d')

TESTPATH = 'D:/Datos de Usuario/cleon/Documents/Mercado Empresas/Data Fuente Comisiones/test/'
inidb = fl.ReadIniFile('C:/Anaconda3/MeScripts/Comisiones4/Config/mydbconfig.ini')
dbparser = inidb.readFile()

dbmanager = dhdb.DbDataProcess(str(periodo))
dbmanager.setParser(dbparser)

In [13]:
def textToDateConverter(df, date_cols):
# Text cols to dates converter    
    for datevalue in date_cols:
        df[datevalue] = pd.to_datetime(df[datevalue],'%Y-%m-%d', dayfirst = True)
    
    return df

In [9]:
# Data Wrangling Ventas y Deacs Voz
# ********************************************************
# Equipos quincenales : actualizar
start_time = time.time()

# PROCESO DE REPORTERIA SOBRE ACTIVACIONES VOZ
#********************************************

HALF_MONTHTLY_TEAMS = ['FALCO','MICHA','KRATOS','KAIZEN']
DATE_COLUMNS = ['FECHA_PROCESO','FEC_ACTIV','FEC_DESACTIVA']
ORDER_COLUMN = ['PERIODO_ACTIVACION','GERENCIA1','GERENCIA2','DEPARTAMENTO','ZONAVENTA','SUPERVISORKAM','VENDEDOR',
                'FECHA_PROCESO','ESTADO']

CHANNEL_FILTER1 = ['PEQUEÑAS Y MEDIANAS EMPRESAS','PLATAFORMAS COMERCIALES REGIONES']
CHANNEL_FILTER2 = ['CORPORACIONES','GRANDES CLIENTES','VENTA REGIONAL EMPRESA']
CHANNEL_FILTER3 = ['HOGAR']

DROP_COLUMNS = ['GERENCIA2','ZONAVENTA','VENDEDOR_OTRO']
DROP_COLUMNS2 = ['NEW_GERENCIA2','NEW_ZONAVENTA','VENDEDOR_OTRO']
RENAME_COLUMNS ={'NEW_GERENCIA2':'GERENCIA2','NEW_ZONAVENTA':'ZONAVENTA','COMISION_UNITARIA':'COMISION_UNITARIA_DB'}

COLS = ['PERIODO_ACTIVACION','GERENCIA1','GERENCIA2','NEW_GERENCIA2','ZONAVENTA','NEW_ZONAVENTA','DEPARTAMENTO','SUPERVISORKAM',
        'VENDEDOR','VENDEDOR_OTRO','POSICION_EMPL',
        'FECHA_PROCESO','CANALDEVENTA','CODIGO','RAZON_SOCIAL','CONTRATO','FEC_ACTIV','FEC_DESACTIVA', 'MODELO','TELEFONO',
        'ESTADO','TMCODE','PLAN_TARIFARIO','DIVISION','FECHA_ACTUALIZACION','TIPODOC','DOCUMENTO','GROSSCOMERCIAL',
        'GROSSCOMISION','ACCESS', 
        'ACCESSBOLSA','ACCESSPAQUETE','ACCESSLICENCIA', 'TIPOEQUIPO', 'RUC_10_20','NEWSADDS','TECNOLOGIAEQUIPO', 'NROORDEN', 
        'ACTIVIDAD','NUEVOSEGMENTO','CEDENTE', 'TIPOVENTA','DEPARTAMENTOGROSS','CARTERIZACION','FACTOR_FACTURACIÓN', 
        'DEALERS_REGIONES', 'COMISION_UNITARIA','PRIMA_SEGURO_ENTEL','ACCESSMASVEP','SEGMENTO_ACCESS']

## Revisar los contratos nulos en BD ##

#"""
ventas_ = dbmanager.loadData('View_Ventas')
ventas_ = textToDateConverter(ventas_, DATE_COLUMNS)
ventas_ = ventas_[COLS]

#ventas_.to_csv(TESTPATH + str(periodo) + '_ventas_.csv', index = False) # Punto de Control

filter1 = ventas_['PERIODO_ACTIVACION'] == periodo
filter2 = ventas_['PERIODO_ACTIVACION'] == end_date_dt.month + 201800 # Tener en cuenta diciembre
filter3 = ventas_['ZONAVENTA'].isin(HALF_MONTHTLY_TEAMS)
filter4 = ventas_['FECHA_PROCESO'] <= end_date_dt

ventast_1 = ventas_[filter1]
ventast = ventas_[filter2 & filter3 & filter4]
ventas = ventast_1.append(ventast, ignore_index = True)

stage1 = ventas['NEW_GERENCIA2'].isin(CHANNEL_FILTER2)
stage2 = ventas['GERENCIA1'].isin(CHANNEL_FILTER1)
stage3 = ventas['GERENCIA2'].isin(CHANNEL_FILTER3)

# Filtrando las ventas_gce
#*************************

ventas_gce = ventas[stage1]
# Corrigiendo los códigos de regiones múltiples
filter_reg = ventas_gce['VENDEDOR_OTRO'] != ''
multiple_seller = ventas_gce[filter_reg]['VENDEDOR_OTRO'].values
ventas_gce.loc[filter_reg,'VENDEDOR'] = multiple_seller
ventas_gce.drop(DROP_COLUMNS, axis =1, inplace = True)
ventas_gce = ventas_gce.rename(columns = RENAME_COLUMNS)

ventas_gce.sort(columns = ORDER_COLUMN,ascending = [1,1,1,1,1,1,1,1,1], inplace = True )
ventas_gce.to_csv(TESTPATH + str(periodo) + '_ventas_gce.csv', index = False)

# Filtrando las ventas_gcp
#*************************

ventas_gcp = ventas[stage2 & ~ stage3]
order_col = ORDER_COLUMN.copy()
order_col.pop(0)
ventas_gcp.drop(DROP_COLUMNS2, axis =1, inplace = True)
ventas_gcp.sort(columns = order_col,ascending = [1,1,1,1,1,1,1,1], inplace = True )
ventas_gcp.to_csv(TESTPATH + str(periodo) + '_ventas_gcp.csv', index = False)

#"""

time_consuming = (time.time() - start_time) # In seconds

print("--- %s seconds ---" % time_consuming)

C:\Anaconda3\lib\site-packages\pandas\core\indexing.py:415: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Anaconda3\lib\site-packages\IPython\kernel\__main__.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda3\lib\site-packages\IPython\kernel\__main__.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda3\lib\site-packages\pandas\core\frame.py:2915: SettingWithCopyWarning: 
A

Los registros de la tabla View_Ventas es 47772 registros  
--- 66.24751281738281 seconds ---


In [14]:
# Data Wrangling Ventas Data
# ********************************************************
# Equipos quincenales : actualizar
start_time = time.time()

half_monthly_teams = ['FALCO','MICHA','KRATOS','KAIZEN']
end_date_dt = datetime.strptime(end_date, '%Y-%m-%d')

DATE_COLUMNS_DATA = ['ACTION_DATE']

ORDER_COLUMN_DATA = ['PERIODO_ACTIVACION','GERENCIA1','GERENCIA2','DEPARTAMENTO','ZONAVENTA','SUPERVISORKAM',
                     'VENDEDOR_CROSS_SELLING','ACTION_DATE','ESTADO']

CHANNEL_FILTER1 = ['PEQUEÑAS Y MEDIANAS EMPRESAS','PLATAFORMAS COMERCIALES REGIONES']
CHANNEL_FILTER2 = ['CORPORACIONES','GRANDES CLIENTES','VENTA REGIONAL EMPRESA']
CHANNEL_FILTER3 = ['HOGAR']

DROP_COLUMNS = ['GERENCIA2','ZONAVENTA','VENDEDOR_OTRO']
DROP_COLUMNS2 = ['NEW_GERENCIA2','NEW_ZONAVENTA','VENDEDOR_OTRO']
RENAME_COLUMNS_DATA ={'NEW_GERENCIA2':'GERENCIA2','NEW_ZONAVENTA':'ZONAVENTA'}

COLS_DATA = ['PERIODO_ACTIVACION','GERENCIA1','GERENCIA2','NEW_GERENCIA2','ZONAVENTA','NEW_ZONAVENTA','DEPARTAMENTO',
             'SUPERVISORKAM','VENDEDOR_CROSS_SELLING','VENDEDOR_OTRO','POSICION_EMPL','ACTION_DATE','CANALDEVENTA',
             'CODIGO','RAZON_SOCIAL','CONTRATO','SERVICE','PHONENUMBER','ESTADO','FECHA_ACTUALIZACION','RUC','ACCESS_REAL',
             'TIPO2','GANADO_POR','EQUIPO','CUENTA','OBSERVACIONES']

## Revisar los contratos nulos en BD ##

# Importing, cleaning, filtering and appending == > initial
ventas_ssaa_ = dbmanager.loadData('View_Ventas_SSAA')
ventas_ssaa_ = textToDateConverter(ventas_ssaa_, DATE_COLUMNS_DATA)
ventas_ssaa_ = ventas_ssaa_[COLS_DATA]

ventas_ssaa_.to_csv(TESTPATH + str(periodo) + '_ventas_ssaa_.csv', index = False) # Para Soluciones de Negocio

filter1 = ventas_ssaa_['PERIODO_ACTIVACION'] == periodo
filter2 = ventas_ssaa_['PERIODO_ACTIVACION'] == end_date_dt.month + 201800 # Tener en cuenta diciembre
filter3 = ventas_ssaa_['ZONAVENTA'].isin(half_monthly_teams)
filter4 = ventas_ssaa_['ACTION_DATE'] <= end_date_dt

ventas_ssaa_t_1 = ventas_ssaa_[filter1]
ventas_ssaa_t = ventas_ssaa_[filter2 & filter3 & filter4]
ventas_ssaa = ventas_ssaa_t_1.append(ventas_ssaa_t, ignore_index = True)

stage1 = ventas_ssaa['NEW_GERENCIA2'].isin(CHANNEL_FILTER2)
stage2 = ventas_ssaa['GERENCIA1'].isin(CHANNEL_FILTER1)
stage3 = ventas_ssaa['GERENCIA2'].isin(CHANNEL_FILTER3)

#"""
# Filtrando las ventas_ssaa_gce
#*************************
ventas_ssaa_gce = ventas_ssaa[stage1]
# Corrigiendo los códigos de regiones múltiples
filter_reg = ventas_ssaa_gce['VENDEDOR_OTRO'] != ''
multiple_seller = ventas_ssaa_gce[filter_reg]['VENDEDOR_OTRO'].values
ventas_ssaa_gce.loc[filter_reg,'VENDEDOR'] = multiple_seller
ventas_ssaa_gce.drop(DROP_COLUMNS, axis =1, inplace = True)
ventas_ssaa_gce = ventas_ssaa_gce.rename(columns = RENAME_COLUMNS_DATA)

ventas_ssaa_gce.sort(columns = ORDER_COLUMN_DATA,ascending = [1,1,1,1,1,1,1,1,1], inplace = True )
ventas_ssaa_gce.to_csv(TESTPATH + str(periodo) + '_ventas_ssaa_gce.csv', index = False)

#"""

#"""
# Filtrando las ventas_ssaa_gcp
#*************************

ventas_ssaa_gcp = ventas_ssaa[stage2 & ~ stage3]
ventas_ssaa_gcp.drop(DROP_COLUMNS2, axis =1, inplace = True)
ventas_ssaa_gcp.sort(columns = ORDER_COLUMN_DATA,ascending = [1,1,1,1,1,1,1,1,1], inplace = True )
ventas_ssaa_gcp.to_csv(TESTPATH + str(periodo) + '_ventas_ssaa_gcp.csv', index = False)

#"""

time_consuming = (time.time() - start_time) # In seconds

print("--- %s seconds ---" % time_consuming)

C:\Anaconda3\lib\site-packages\pandas\core\indexing.py:249: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Anaconda3\lib\site-packages\pandas\core\indexing.py:415: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Anaconda3\lib\site-packages\IPython\kernel\__main__.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-vie

Los registros de la tabla View_Ventas_SSAA es 3124 registros  
--- 6.800594091415405 seconds ---


C:\Anaconda3\lib\site-packages\IPython\kernel\__main__.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda3\lib\site-packages\pandas\core\frame.py:2915: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  inplace=inplace, kind=kind, na_position=na_position)
